In [3]:
import pandas as pd
import numpy as np
import re 
import jieba
import matplotlib.pyplot as plt
from gensim import corpora, models
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis


In [2]:
# pip install pyLDAvis

   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.6 MB 1.3 MB/s eta 0:00:03
   - -------------------------------------- 0.1/2.6 MB 1.5 MB/s eta 0:00:02
   - -------------------------------------- 0.1/2.6 MB 1.5 MB/s eta 0:00:02
   - -------------------------------------- 0.1/2.6 MB 1.5 MB/s eta 0:00:02
   - -------------------------------------- 0.1/2.6 MB 1.5 MB/s eta 0:00:02
   - -------------------------------------- 0.1/2.6 MB 1.5 MB/s eta 0:00:02
   -- ------------------------------------- 0.2/2.6 MB 517.2 kB/s eta 0:00:05
   -- ------------------------------------- 0.2/2.6 


[notice] A new release of pip is available: 23.3.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
df = pd.read_excel('text_analysis_weibo_sample.xlsx', index_col=0)

In [5]:
df.head()

,index,标题/微博内容,点赞,转发,评论,账号昵称UID加密,粉丝数,关注数,地域
0,34121,国债：地产行业重磅利好提振风险偏好，期债低开低走 国债期货全线收跌，10年期主力...,0,0,0,e5df796860e68f403bcf9651bab4d42e,0,0,其他
1,40230,#喜迎二十大 忠诚保平安#,0,0,0,6e35cb69ad52f20de5e28197b2e85306,405444,252,广西
2,7714,注意！事关明日教资考试！福建省教育考试院发布补充公告 福建省2022年下半年全国中小学教师...,0,0,0,e6953217442e6c06a7af23eee5e185f2,53264,2177,福建
3,27378,近日，“千年大计”雄安新区迎来五周岁生日。从“一张白纸...,0,0,0,NaN,0,0,北京
4,15435,樊振东牛逼！,0,0,0,344af41eac516375c04dee6325e763cc,8,51,山东


# 文本分词 + 去除非中文字符

In [6]:
def clean_text(text):
    text = re.sub(r'[^\u4e00-\u9FA5]+', '', text)     #去除非中文字符
    words = jieba.lcut(text)
    words = [word for word in words if len(word) >= 2]
    return ' '.join(words)

In [8]:
df['分词'] = df['标题/微博内容'].apply(clean_text)
df['分词'] = df['分词'].apply(lambda x: x.split())
df['分词'].head(10)

0    [国债, 地产, 行业, 重磅, 利好, 提振, 风险, 偏好, 期债, 低开, 国债, 期...
1                                   [喜迎, 二十大, 忠诚, 保平安]
2    [注意, 事关, 明日, 教资, 考试, 福建省, 教育, 考试院, 发布, 补充, 公告,...
3    [近日, 千年, 大计, 雄安, 新区, 迎来, 五周岁, 生日, 一张白纸, 塔吊, 林立...
4                                                [樊振东]
5                              [德州, 同城会, 不能, 心向往之, 哼哼]
6       [严惩, 弄虚作假, 玩弄, 全国, 网民, 罪犯, 克克, 你们, 厉害, 两个, 词条]
7                                                 [关注]
8    [怎么, 看到, 这条, 哈哈哈哈, 哈哈哈哈, 哈哈哈, 许昕, 这个, 团队, 半天, ...
9    [截至, 目前, 南京, 东莞, 已有, 万亿, 俱乐部, 城市, 晒出, 上半年, 成绩单...
Name: 分词, dtype: object

# 构造LDA模型

In [9]:
import gensim

In [10]:
dictionary = corpora.Dictionary(df['分词'])
corpus = [dictionary.doc2bow(text) for text in df['分词']]

In [11]:
# 训练模型
lda_model = gensim.models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

In [12]:
# 看主题内容
topic = lda_model.print_topics(num_topics=5, num_words=10)

In [13]:
topic

[(0,
  '0.009*"加息" + 0.008*"通胀" + 0.007*"市场" + 0.007*"预期" + 0.006*"报告" + 0.006*"放缓" + 0.005*"美国" + 0.005*"同比" + 0.005*"数据" + 0.004*"政策"'),
 (1,
  '0.011*"郑州" + 0.010*"经济" + 0.009*"数字" + 0.009*"中国" + 0.009*"城市" + 0.009*"建设" + 0.008*"发展" + 0.006*"亿元" + 0.005*"增长" + 0.004*"重要"'),
 (2,
  '0.015*"考生" + 0.009*"考试" + 0.008*"考场" + 0.008*"开户" + 0.007*"期货" + 0.006*"考点" + 0.005*"我们" + 0.005*"原油" + 0.005*"核酸" + 0.004*"自己"'),
 (3,
  '0.016*"万吨" + 0.010*"市场" + 0.009*"库存" + 0.007*"需求" + 0.007*"下降" + 0.007*"价格" + 0.006*"现货" + 0.006*"环比" + 0.006*"预期" + 0.006*"元吨"'),
 (4,
  '0.021*"电子" + 0.009*"亿元" + 0.008*"市场" + 0.007*"中国" + 0.007*"重庆" + 0.007*"政策" + 0.006*"增长" + 0.005*"银行" + 0.005*"出口" + 0.004*"公司"')]

In [14]:
def infer_topics(model, document):
    # 将文档转换为词袋（Bag of Words）格式，供LDA模型使用
    bow = dictionary.doc2bow(document)
    # 使用训练好的LDA模型获取该文档的主题分布
    topics = lda_model.get_document_topics(bow)
    return topics

documents = df['分词'].values.tolist()
for i, doc in enumerate(documents[:5]):
        # 打印文档编号和内容
    print(f"Document {i}: {doc}")
    doc_topics = infer_topics(lda_model, doc)
    print(f"Topics: {doc_topics}")
    print("\n")

Document 0: ['国债', '地产', '行业', '重磅', '利好', '提振', '风险', '偏好', '期债', '低开', '国债', '期货', '全线', '收跌', '年期', '主力', '合约', '年期', '主力', '合约', '年期', '主力', '合约', '跌三大', '主力', '合约', '一年', '收盘', '新低', '行情', '解读', '公开市场', '方面', '央行', '开展', '亿元', '天期', '回购', '操作', '中标', '利率', '亿元', '回购', '到期', '因此', '当日', '投放', '亿元', '资金面', '方面', '央行', '公开市场', '回购', '继续', '加力', '不过', '银行', '市场', '周二', '资金', '供给', '趋于', '收敛', '七天', '回购', '加权', '利率', '进一步', '上行', '长期', '资金', '方面', '全国', '主要', '股份制', '银行', '一年期', '同业', '存单', '报价', '维持', '水平', '不过', '目前', '尚无', '成交量', '配合', '房地产', '行业', '再度', '迎来', '重磅', '利好', '证监会', '宣布', '涉房', '企业', '股权', '融资', '方面', '调整', '优化', '五项', '措施', '自即日起', '施行', '随着', '证监会', '支持', '房企', '股权', '融资', '第三支', '正式', '发射', '信贷', '债券', '股权', '三个', '融资', '渠道', '三箭', '齐发', '合力', '推动', '房地产', '融资', '第三支', '快速', '落地', '房地产', '行业', '注入', '大规模', '增量', '资金', '资产', '促进', '行业', '平稳', '健康', '发展', '短短', '时间', '政策', '端三箭', '齐发', '强势', '托底', '提振', '各方', '信心', '看好', '行业', '信用', '复苏', '以及', '因城', '施策', '空间', '逐步', 

# 可视化

In [15]:
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary)
display(pyLDAvis.display(lda_vis))